### Processing as Decorating II

In [ ]:
"""development on 1.2 example, composition of transformations on one pass"""
import nographs as nog
from graphviz import Digraph
from decorator import decorator

In [ ]:
dict_1= {0:{1,2}, 1:{2,0}, 2:{0,1}, }

# dict_2= {0:{1,2,4}, 1:{3,5}, 2:{3,6}, 3:{37}, 37:{7}, 4:{5,6}, 5:{57}, 57:{7},
#          6:{67}, 67:{7}, 7:{} }

def forward(vert, _):
    """basic `next vertices`"""
    return dict_1[vert]

In [ ]:
# https://www.graphviz.org/doc/info/attrs.html
node_style ={"style": 'filled',            "shape": 'box',
             "align": 'left',              "fontsize": '12',
             "ranksep":'0.01',             "height":'0.02',   "width":'0.04',}
gr_style = {"linelength": '16', "rankdir": 'LR'}#landscape="True",layout="neato"

def adapter_graphviz( to_deco ):
    """translation to GraphViz visitor"""
    dot = Digraph( node_attr = node_style, graph_attr = gr_style, )

    def wrapper_bg( vert, _trav ):
        """visitor wrapper"""
        dot.node( str(hash( vert )), str( vert ) )
        for result in to_deco( vert, _trav):
            dot.edge( str(hash( vert )), str(hash( result )), )
            yield result
    wrapper_bg.shared = getattr( to_deco,"shared",{})
    wrapper_bg.shared["dot"] = dot
    return wrapper_bg

In [ ]:
class EdgeSplitter:
    """class-like decorator, inserting vertex in every edge"""
    def __init__(self, to_deco):
        self.to_deco = to_deco
        self.shared = getattr( to_deco,"shared",{}).copy()
        self.newbies = set()
        # update_wrapper(self, to_deco, updated=())
    def __call__(self, vert, _trav):
        if vert in self.newbies:
            yield vert[1]
        else:
            for result in self.to_deco(vert, _trav):
                # creating new vertex at the middle of edge,
                # new id is tuple of edge's ends
                self.newbies.add((vert, result))
                yield (vert, result)

# test with reenterability
@adapter_graphviz
@EdgeSplitter
@EdgeSplitter
def double_splitted(vert, _):
    """modified `next vertices`"""
    return dict_1[vert]

In [ ]:
trav_forward = nog.TraversalBreadthFirst( double_splitted )
trav_forward.start_from( 0, build_paths=True )
str( list(trav_forward) )

In [ ]:
double_splitted.shared["dot"].attr(size='8,4')
# double_splitted.shared["dot"].attr( layout="neato")
double_splitted.shared["dot"]              # pylint: disable=pointless-statement

In [ ]:
@decorator
def identity_decorator( wrapped, item, _trav ):
    """Starting point to make your brand new one"""
    yield from wrapped( item, _trav )

@decorator
def filter_big( wrapped, vert, _trav ):
    """Such as"""
    yield from filter( lambda vert:vert<100, wrapped( vert, _trav ) )